In [ ]:
from pathlib import Path
from dataclasses import dataclass
import math

import face_alignment
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
matplotlib.rcParams['figure.dpi'] = 300
import cv2
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np
import pandas as pd

In [ ]:
# from importlib import reload
# reload(face_alignment)

In [ ]:
# BASE_DIR = Path(__file__).parent.parent
# BASE_DIR = Path.cwd().
DRIVE_NAME = "F:\\"
DATA_DIR = Path(DRIVE_NAME).joinpath('data')
IMAGE_DIR = DATA_DIR.joinpath('images')
DUMP_DIR = DATA_DIR.joinpath('dump')
MODEL_DIR = DATA_DIR.joinpath('models')
TEST_DIR = DATA_DIR.joinpath('test')
# for c in DATA_DIR.iterdir(): print(c)
# test_dir = IMAGE_DIR.joinpath('2020-08-17')
# test_dir = IMAGE_DIR.joinpath('2020-06-25')
# test_dir = IMAGE_DIR.joinpath('2012-06-11')

class FaceDetection:
    w_1: int
    h_1: int
    w_2: int
    h_2: int
        
    side_length: int
    confidence: float

    def __init__(self, face_detection: list, multiplier: float):
        w_1 = face_detection[0]
        h_1 = face_detection[1]
        w_2 = face_detection[2]
        h_2 = face_detection[3]
                
        height = h_2 - h_1
        width = w_2 - w_1
        height_center = h_2 - (h_2 - h_1)/2
        width_center = w_2 - (w_2 - w_1)/2
        if width > height:
            self.side_length = width * multiplier
        else:
            self.side_length = height * multiplier
        self.confidence = face_detection[4]
        self.h_1 = math.floor(height_center - self.side_length/2)
        self.h_2 = math.ceil(height_center + self.side_length/2)
        self.w_1 = math.floor(width_center - self.side_length/2)
        self.w_2 = math.ceil(width_center + self.side_length/2)
        
@dataclass
class Predtype:
    list_slice: slice
    color: tuple

pred_types = {
    'face': Predtype(slice(0, 17), (0.682, 0.780, 0.909, 0.5)),
    'eyebrow1': Predtype(slice(17, 22), (1.0, 0.498, 0.055, 0.4)),
    'eyebrow2': Predtype(slice(22, 27), (1.0, 0.498, 0.055, 0.4)),
    'nose': Predtype(slice(27, 31), (0.345, 0.239, 0.443, 0.4)),
    'nostril': Predtype(slice(31, 36), (0.345, 0.239, 0.443, 0.4)),
    'eye1': Predtype(slice(36, 42), (0.596, 0.875, 0.541, 0.3)),
    'eye2': Predtype(slice(42, 48), (0.596, 0.875, 0.541, 0.3)),
    'lips': Predtype(slice(48, 60), (0.596, 0.875, 0.541, 0.3)),
    'teeth': Predtype(slice(60, 68), (0.596, 0.875, 0.541, 0.4))
}


In [ ]:
fa = face_alignment.FaceAlignment(
    face_alignment.LandmarksType._3D,
    # face_alignment.LandmarksType._2D,
    face_detector='sfd',
    # face_detector='blazeface',
    flip_input=False,
    device='cpu',
    verbose=True,
)

In [ ]:
sfd_resize_p = 480

df_as_list_of_dicts = []
for image_dir in TEST_DIR.iterdir():
    print("----- Processing images in {}".format(image_dir))
    files = list(image_dir.iterdir())
    f_i = 0
    f_n = len(files)
    for f in files:
        # somehow check if directory has already been processed
        if(False):
            print("!!!!! Directory already processed")
            break
        if f.suffix in ['.jpg', '.png', '.jfif']:
            f_i += 1
            print("--- Processing {}/{}, {}".format(
                f_i,
                f_n,
                f.name
            ))
            img = io.imread(f)

            # standard format is (WIDTH x HEIGHT)
            # but img.shape returns it in opposite order
            height, width, _ = img.shape
            
            if height > width:
                # Vertical image
                multiplication_factor = sfd_resize_p/width
                dim = (sfd_resize_p, int(height*multiplication_factor))
            else:
                # Horizontal image
                multiplication_factor = sfd_resize_p/height
                dim = (int(width*multiplication_factor), sfd_resize_p)

            resized_img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            landmarks, detections = fa.get_landmarks_from_image(resized_img)
            if landmarks is None and detections is None:
                continue



            print("--- {} faces detected".format(len(detections)))
            # for detection in detections:
            for index, detection in enumerate(detections):
                print("- Confidence: {}".format(detection[4]))

                # detection[0] = detection[0]/multiplication_factor
                # detection[1] = detection[1]/multiplication_factor
                # detection[2] = detection[2]/multiplication_factor
                # detection[3] = detection[3]/multiplication_factor


                df_as_list_of_dicts.append({
                    'img_path': f,
                    'detection': np.array(detection)/multiplication_factor,
                    'landmarks': np.array(landmarks[index])/multiplication_factor,
                })
                # df_as_list_of_dicts.append({
                #     'img_path': f,
                #     'fd_y1': detection[0],
                #     'fd_x1': detection[1],
                #     'fd_y2': detection[2],
                #     'fd_x2': detection[3],
                #     'confidence': detection[4],
                #     'landmarks': landmarks[index],
                # })
df = pd.DataFrame.from_dict(df_as_list_of_dicts)

In [ ]:
# df["landmarks"][0]
df

In [ ]:
img_path = None
plot_style = dict(
    marker='.',
    markersize=1,
    linestyle='none',
    # linewidth=2,
)

# Analyze images in dataframe
for index, row in df.iterrows():
    detection = FaceDetection(row['detection'], multiplier=1.25)


    # Draw image if looking at a different image (or first image) than the last image
    if img_path is None or img_path != row['img_path']:
        img_path = row['img_path']
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])
        ax.imshow(io.imread(img_path))
        ax.axis('off')
        print("Confidence: {}".format(detection.confidence))

    # Draw bounding box
    rect = patches.Rectangle(
        (detection.w_1, detection.h_1),
        detection.side_length,
        detection.side_length,
        linewidth=1,
        edgecolor='g',
        facecolor='none',
    )
    ax.add_patch(rect)
    plt.plot(detection.w_1, detection.h_1, 'ro')
    plt.plot(detection.w_1, detection.h_2, 'go')
    plt.plot(detection.w_2, detection.h_1, 'bo')
    plt.plot(detection.w_2, detection.h_2, 'yo')



    # Draw 2D landmarks

# for img_path, faces in img_preds.items():
#     # print(img_path)
#     # print(len(value))
#     # print("\n")
#     fig = plt.figure()
#     ax = fig.add_axes([0, 0, 1, 1])
#     ax.imshow(io.imread(img_path))
    # ax.axis('off')



    # for index, landmark in enumerate(row['landmarks']):
    # landmark = 
    # print(landmark)
    # ax = fig.add_subplot(1, len(faces), index + 1)
    for pred_type in pred_types.values():
        # print(pred)
        # print(type(pred))
        
        ax.plot(
            row['landmarks'][pred_type.list_slice, 0],
            row['landmarks'][pred_type.list_slice, 1],
            color=pred_type.color,
            **plot_style,
        )

        
        


In [ ]:
img = io.imread(img_path)

In [ ]:
vggface = VGGFace(
    # model='resnet50',
    model='senet50',
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
)

In [ ]:
vgg_resize_p = 224
column_as_list_of_dicts = []
for index, row in df.iterrows():
    
    img = io.imread(row['img_path'])


    # fig = plt.figure()
    # ax = fig.add_axes([0, 0, 1, 1])
    # ax.imshow(img)
    # ax.axis('off')
    

    detection = FaceDetection(row['detection'], multiplier=1.25)
    # print(img.shape)
    # print(detection.h_1, detection.w_1)
    # print(detection.h_2, detection.w_2)
    # print(detection.side_length)

    face_img = img[
        detection.h_1:detection.h_2,
        detection.w_1:detection.w_2,
    ]
    # print(face_img.shape)
    
    
    if detection.side_length > vgg_resize_p:
        interp_method = cv2.INTER_AREA
    else:
        interp_method = cv2.INTER_CUBIC
    face_img = cv2.resize(face_img, (vgg_resize_p, vgg_resize_p), interpolation=interp_method)


    # fig = plt.figure()
    # ax = fig.add_axes([0, 0, 1, 1])
    # ax.imshow(face_img)
    # ax.axis('off')
    

    samples = preprocess_input(np.expand_dims(face_img.astype('float32'), axis=0), version=2)
    embedding = vggface.predict(samples)
    column_as_list_of_dicts.append({
        "embedding": embedding
    })


    # add code to save stuff
    # add code from extra_code to check bbox off image edges
        


    
    



In [ ]:

sum(column_as_list_of_dicts[4]['embedding'][0])
# type(column_as_list_of_dicts[0]['embedding'][0])